In [7]:
import pandas as pd
import os

In [8]:
my_file = os.path.join('data', 'available_datasets.tsv')
available_datasets = pd.read_csv(my_file, sep="\t")

In [9]:
available_datasets

,chemistry,reference,dataset_name,link,data_url,MD5,delete_fastq,feature_barcode,library_csv,quant_link
0,v3,human2020A,"500 Human PBMCs, 3' LT v3.1, Chromium Controller",https://www.10xgenomics.com/resources/datasets...,https://cf.10xgenomics.com/samples/cell-exp/6....,5f080c6082f11ea9fc6448482e6fb590,1,NaN,NaN,https://umd.box.com/shared/static/tg919re5gd4k...
1,v3,human2020A,"500 Human PBMCs, 3' LT v3.1, Chromium X",https://www.10xgenomics.com/resources/datasets...,https://cf.10xgenomics.com/samples/cell-exp/6....,5b36a7bfda36a7093adc8e30c3fa92c8,1,NaN,NaN,https://umd.box.com/shared/static/lrl68q2lz0lt...
2,v3,human2020A,1k PBMCs from a Healthy Donor (v3 chemistry),https://www.10xgenomics.com/resources/datasets...,https://cf.10xgenomics.com/samples/cell-exp/3....,265ebe8f77ad90db350984d9c7a59e52,1,NaN,NaN,https://umd.box.com/shared/static/wrn19wsmkem1...
3,v3,mm10-2020A,10k PBMCs from a Healthy Donor (v3 chemistry),https://www.10xgenomics.com/resources/datasets...,https://s3-us-west-2.amazonaws.com/10x.files/s...,e0021592e209642d71f5dc420cf4c5c0,1,NaN,NaN,https://umd.box.com/shared/static/01j9574g1yd9...
4,v3,human2020A,"10k Human PBMCs, 3' v3.1, Chromium X",https://www.10xgenomics.com/resources/datasets...,https://s3-us-west-2.amazonaws.com/10x.files/s...,43ea77ed6f860597c568e9ff819f0504,1,NaN,NaN,https://umd.box.com/shared/static/jvvzacmo98vx...
5,v3,human2020A,"10k Human PBMCs, 3' v3.1, Chromium Controller",https://www.10xgenomics.com/resources/datasets...,https://s3-us-west-2.amazonaws.com/10x.files/s...,05cccabbad1b94c83f67c67b15b07e7e,1,NaN,NaN,https://umd.box.com/shared/static/5dzu2tw8nz9t...
6,v3,human2020A,10k Peripheral blood mononuclear cells (PBMCs)...,https://www.10xgenomics.com/resources/datasets...,https://s3-us-west-2.amazonaws.com/10x.files/s...,38d2d253f8537d3c39aaa5832da39ad3,1,NaN,NaN,https://umd.box.com/shared/static/iol9bxiv740x...
7,v3,human2020A,10k Peripheral blood mononuclear cells (PBMCs)...,https://www.10xgenomics.com/resources/datasets...,https://s3-us-west-2.amazonaws.com/10x.files/s...,bd61d65af63508368c57f137090f32b9,1,NaN,NaN,https://umd.box.com/shared/static/5dzu2tw8nz9t...
8,v3,human2020A,"20k Human PBMCs, 3' HT v3.1, Chromium X",https://www.10xgenomics.com/resources/datasets...,https://s3-us-west-2.amazonaws.com/10x.files/s...,650201ca6bd1af3f984a629da318d23b,1,NaN,NaN,https://umd.box.com/shared/static/c609sk8w6cbn...
9,v3,human2020A,PBMCs from EDTA-Treated Blood Collection Tubes...,https://www.10xgenomics.com/resources/datasets...,https://cf.10xgenomics.com/samples/cell-exp/6....,d07c96c9f224d75967759db6be54d635,1,NaN,NaN,https://umd.box.com/shared/static/imedrs558dx4...


In [19]:
class processed_quant:
    """
    A class stores the information of the quantification
    result of a processed dataset
    """
    def __init__(self, available_datset):
        self.chemistry = available_datset["chemistry"]
        self.reference = available_datset["reference"]	
        self.dataset_name = available_datset["dataset_name"]
        self.link = available_datset["link"]
        self.data_url = available_datset["data_url"]
        self.MD5 = available_datset["MD5"]
        self.delete_fastq = available_datset["delete_fastq"]
        self.feature_barcode = available_datset["feature_barcode"]
        self.library_csv = available_datset["library_csv"]
        self.quant_link = available_datset["quant_link"]
        self.quant_path = nan
        self.tar_path = nan


In [17]:
curr_dataset = available_datasets.iloc[dataset_id,:]

In [18]:
curr_dataset["chemistry"]

'v3'

In [20]:
a = None

In [21]:
a is not None

False

In [22]:
import os

In [30]:
class processed_quant:
    """
    A class stores the information of the quantification
    result of a processed dataset
    """
    def __init__(self, available_datasets, dataset_id):
        available_datset = available_datasets.iloc[dataset_id-1,:]
        self.dataset_id = dataset_id
        self.chemistry = available_datset["chemistry"]
        self.reference = available_datset["reference"]	
        self.dataset_name = available_datset["dataset_name"]
        self.link = available_datset["link"]
        self.data_url = available_datset["data_url"]
        self.MD5 = available_datset["MD5"]
        self.delete_fastq = available_datset["delete_fastq"]
        self.feature_barcode = available_datset["feature_barcode"]
        self.library_csv = available_datset["library_csv"]
        self.quant_link = available_datset["quant_link"]
        self.quant_path = None
        self.tar_path = None
        self.anndata = None

    def fetch_quant(self, fetch_dir="processed_quant", force=False, delete_tar=True, quiet = False):
        if not self.check_validity():
            raise ValueError("Incomplete class object, cannot proceed")

        say(quiet, f"Processing dataset #{self.dataset_id}")

        # check whether quant path exist
        # remove it if needed
        if self.quant_path is not None:
            quant_parent_dir = os.path.dirname(self.quant_path)
            if os.path.exists(quant_parent_dir):
                if (not force):
                    say(quiet, f"  - quant path exists:\n    {quant_parent_dir}")
                    say(quiet, f"  - if want to overwrite it, pass force=True")
                    return
                else:
                    shutil.rmtree(self.quant_path)

        quant_parent_dir = os.path.join(fetch_dir, 
                                        f"{self.dataset_id}")

        # check whether tar file exist,
        # download it if needed
        if (self.tar_path is not None) and \
            os.path.exists(self.tar_path) and \
            (not force):
                    say(quiet, f"  - using existing tar file:\n    {self.tar_path}")
                    say(quiet, f"  - if want to fetch it again, pass force=True")
        else:
            # folder for (temporarily) storing tar files.
            tar_dir = os.path.join(fetch_dir, "datasets_tar")
            if not os.path.exists(tar_dir):
                os.makedirs(tar_dir)

            # update tar path
            self.tar_path = os.path.join(tar_dir,
                                "".join([self.dataset_id, ".tar"]))

            say(quiet, "  - Downloading quant result")
            urllib.request.urlretrieve(self.quant_link, self.tar_path)

        # decompress the downloaded tar files
        say(quiet, "  - Decompressing quant result")
        tf = tarfile.open(self.tar_path)
        tf.extractall(quant_parent_dir)
        self.quant_path = os.path.join(quant_parent_dir, 
                                        next(os.walk(quant_parent_dir))[1][0])

        if delete_tar:
            self.tar_path = nan

        # delete tar if needed
        if delete_tar:
            say(quiet, "Removing downloaded temp tar folder")
            shutil.rmtree(tar_dir)

        say(quiet, "Done")
        return


    def check_validity(self):
        if self.quant_link is None or \
            self.dataset_id is None or \
            self.chemistry is None or \
            self.reference is None or \
            self.dataset_name  is None or \
            self.link is None or \
            self.data_url is None or \
            self.MD5 is None or \
            self.delete_fastq is None or \
            self.feature_barcode is None or \
            self.library_csv is None or \
            self.quant_link is None:
            return False
        return True


In [31]:
a = processed_quant(available_datasets, 3)

In [33]:
print(a)

In [28]:
os.path.parent(os.path.join("a", "b"))

AttributeError: module 'posixpath' has no attribute 'parent'

In [36]:
print("absdc")
print("absdc")

absdc
absdc


In [40]:
"basdg.tar"[:-4]

'basdg'

In [43]:
if [1]:
    print("OK")

OK


In [48]:
a = {1: 1, 2:3}

In [49]:
a[2]

3

In [50]:
a[3] = 5


In [51]:
a

{1: 1, 2: 3, 3: 5}